In [ ]:
#!/usr/bin/env python

# Copyright (c) 2019 Computer Vision Center (CVC) at the Universitat Autonoma de
# Barcelona (UAB).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import random
import time

In [ ]:
actor_list = []

# In this tutorial script, we are going to add a vehicle to the simulation
# and let it drive in autopilot. We will also create a camera attached to
# that vehicle, and save all the images generated by the camera to disk.

In [ ]:
# 시뮬레이터에 요청을 보낼 클라이언트를 생성. 
# 여기서는 시뮬레이터가 포트 2000의 로컬 호스트에서 요청을 수신 가정.

client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [ ]:
# 클라이언트가 생기면 현재 실행 중인 월드를 검색할 수 있다.
world = client.get_world()

In [ ]:
# pedestrian code

world.set_pedestrians_seed(1235)
ped_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))

ped_start_location = carla.Location(x=-34.3, y=2.7, z=0.2)
ped_start_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
start_trans = carla.Transform(ped_start_location, ped_start_rotation)

ped_dst_location = carla.Location(x=-59.3, y=2.7) #z=0.134872
ped_dst_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
dst_trans = carla.Transform(ped_dst_location, ped_dst_rotation)

ped = world.spawn_actor(ped_bp, start_trans)
walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
controller = world.spawn_actor(walker_controller_bp, carla.Transform(), ped)
controller.start()
controller.set_max_speed(1.7)

# adding an actor to an actor list
actor_list.append(ped)
actor_list.append(controller)

while True:
    x = ped.get_location().x
    y = ped.get_location().y        # restrict Unexpected movement

    if x < -61.0 or y < 0.5:
        controller.go_to_location(ped_start_location)
    elif x > -32.0 or y < 0.5:
        controller.go_to_location(ped_dst_location)


In [ ]:
client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])

# ped.destroy()
# controller.destroy()